In [2]:
# Importing dependencies and libraries
import numpy as np
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import os
import datetime

from config import api_key


In [36]:
# Looking at stock performance by industry relevant indexes

# Variable initialization
close_daily_price =[]
ticker_date = []
delta = datetime.timedelta(days=1)

# Search parameters
industry_index = ['XPH', 'PCM']
start_date = datetime.datetime(2019,8,1)
end_date = datetime.datetime(2020,7,31)
base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'

# For loop to create the date column and create the results dataframe
while start_date <= end_date:
    
    ticker_date.append(start_date)
    
    # Building the results dataframe
    ticker_value_df = pd.DataFrame({'Date': ticker_date})
    
    start_date += delta
                                    
# Resetting start date
start_date = datetime.datetime(2019,8,1)
                                    
# For loop to make the call for every industry index and retrieving the data
for ticker in industry_index:
    
    # Building the search url
    url = f'{base_url}&symbol={ticker}&outputsize=full&apikey={api_key}'
    
    # Making the API call
    response = requests.get(url)
    response_json = response.json()
    
    #Progress tracker
    print (f'Processing {ticker} with code {response.status_code}')
    
    # Retrieving daily closing prices
    while start_date <= end_date:
    
        # Obtaining date as string
        string_date = start_date.strftime("%Y-%m-%d")

        # Exception for dates with no operation
        try:
            close_daily_price.append(float(response_json['Time Series (Daily)'][string_date]["4. close"]))
        except (KeyError):
            close_daily_price.append(0)  
        
        start_date += delta
                                    
    # Adding the current ticker to the dataframe
    ticker_value_df[ticker] = pd.Series(close_daily_price)
    close_daily_price.clear()
    start_date = datetime.datetime(2019,8,1)



        

Processing XPH with code 200
Processing PCM with code 200


In [37]:
# Building the results dataframe

ticker_value_df

,Date,XPH,PCM
0,2019-08-01,37.95,11.25
1,2019-08-02,37.69,11.37
2,2019-08-03,0.00,0.00
3,2019-08-04,0.00,0.00
4,2019-08-05,36.61,11.27
...,...,...,...
361,2020-07-27,43.39,9.36
362,2020-07-28,43.06,9.34
363,2020-07-29,43.26,9.38
364,2020-07-30,43.31,9.45


In [41]:
clean_df=ticker_value_df.loc[ticker_value_df["XPH"]!=0].reset_index(drop=True)
clean_df

,Date,XPH,PCM
0,2019-08-01,37.95,11.25
1,2019-08-02,37.69,11.37
2,2019-08-05,36.61,11.27
3,2019-08-06,36.72,11.39
4,2019-08-07,36.77,11.40
...,...,...,...
248,2020-07-27,43.39,9.36
249,2020-07-28,43.06,9.34
250,2020-07-29,43.26,9.38
251,2020-07-30,43.31,9.45


In [42]:
clean_df.describe()

,XPH,PCM
count,253.000000,253.000000
mean,40.601482,10.384289
std,3.990225,1.317334
min,31.560000,6.240000
25%,37.300000,9.380000
50%,41.010000,11.040000
75%,43.410000,11.430000
max,47.910000,12.020000


In [43]:
first_price=ticker_value_df.loc[clean_df["Date"]=='2019-08-01',:]
first_price

last_price=ticker_value_df.loc[clean_df["Date"]=='2020-07-31',:]
last_price

ava_price_change=(last_price-first_price)/(last_price)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [7]:
clean_df.describe()

<ipython-input-7-a6f50b0cde66>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  clean_df.describe()


,Date,XPH,PAU,PBK,PCEQ,PCM,PDFS,PEN,PFSR,PHR
count,253,253,253,253,253,253,253,253,253,253
unique,253,232,232,232,232,232,232,232,232,232
top,2020-03-13 00:00:00,43.1300,43.1300,43.1300,43.1300,43.1300,43.1300,43.1300,43.1300,43.1300
freq,1,2,2,2,2,2,2,2,2,2
first,2019-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2020-07-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
